In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Załaduj zbiór danych Iris
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Definicja sieci neuronowej
class MLP(nn.Module):
    def __init__(self, input_size=4, hidden_size=10, output_size=3):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Funkcja oceny (fitness)
def fitness(weights, model, train_loader, device):
    idx = 0
    for param in model.parameters():
        numel = param.numel()
        param.data = weights[idx:idx+numel].view(param.size()).to(device)
        idx += numel
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(train_loader.dataset)

# Algorytm DEAW
def train_deaw(model, train_loader, device, NP=50, F=0.5, CR=0.9, max_generations=100, initial_lower=-1.0, initial_upper=1.0):
    num_weights = sum(p.numel() for p in model.parameters())
    lower_bounds = np.full(num_weights, initial_lower)
    upper_bounds = np.full(num_weights, initial_upper)
    population = np.random.uniform(initial_lower, initial_upper, (NP, num_weights))
    fitnesses = np.array([fitness(torch.tensor(p, dtype=torch.float32), model, train_loader, device) for p in population])

    for generation in range(max_generations):
        for i in range(NP):
            candidates = [j for j in range(NP) if j != i]
            a, b, c = np.random.choice(candidates, 3, replace=False)
            v = population[a] + F * (population[b] - population[c])

            for j in range(num_weights):
                if v[j] < lower_bounds[j]:
                    lower_bounds[j] *= 2
                    v[j] = lower_bounds[j]
                elif v[j] > upper_bounds[j]:
                    upper_bounds[j] *= 2
                    v[j] = upper_bounds[j]

            u = np.copy(population[i])
            j_rand = np.random.randint(0, num_weights)
            for j in range(num_weights):
                if np.random.rand() < CR or j == j_rand:
                    u[j] = v[j]

            u_tensor = torch.tensor(u, dtype=torch.float32)
            loss_u = fitness(u_tensor, model, train_loader, device)

            if loss_u < fitnesses[i]:
                population[i] = u.copy()
                fitnesses[i] = loss_u

    best_idx = np.argmin(fitnesses)
    best_weights = torch.tensor(population[best_idx], dtype=torch.float32)
    idx = 0
    for param in model.parameters():
        numel = param.numel()
        param.data = best_weights[idx:idx+numel].view(param.size()).to(device)
        idx += numel

# Użycie
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
train_deaw(model, train_loader, device)

In [2]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
accuracy = evaluate_model(model, test_loader, device)
print(f"Dokładność na zbiorze testowym: {accuracy:.2f}%")

Dokładność na zbiorze testowym: 96.67%


In [3]:
def train_de(model, train_loader, device, NP=50, F=0.5, CR=0.9, max_generations=100, initial_lower=-1.0, initial_upper=1.0):
    num_weights = sum(p.numel() for p in model.parameters())
    population = np.random.uniform(initial_lower, initial_upper, (NP, num_weights))
    fitnesses = np.array([fitness(torch.tensor(p, dtype=torch.float32), model, train_loader, device) for p in population])

    for generation in range(max_generations):
        for i in range(NP):
            candidates = [j for j in range(NP) if j != i]
            a, b, c = np.random.choice(candidates, 3, replace=False)
            v = population[a] + F * (population[b] - population[c])

            # Stałe ograniczenie: ograniczamy wartości do zakresu
            v = np.clip(v, initial_lower, initial_upper)

            u = np.copy(population[i])
            j_rand = np.random.randint(0, num_weights)
            for j in range(num_weights):
                if np.random.rand() < CR or j == j_rand:
                    u[j] = v[j]

            u_tensor = torch.tensor(u, dtype=torch.float32)
            loss_u = fitness(u_tensor, model, train_loader, device)

            if loss_u < fitnesses[i]:
                population[i] = u.copy()
                fitnesses[i] = loss_u

    best_idx = np.argmin(fitnesses)
    best_weights = torch.tensor(population[best_idx], dtype=torch.float32)
    idx = 0
    for param in model.parameters():
        numel = param.numel()
        param.data = best_weights[idx:idx+numel].view(param.size()).to(device)
        idx += numel


In [4]:
model_de = MLP().to(device)
train_de(model_de, train_loader, device)
accuracy_de = evaluate_model(model_de, test_loader, device)
print(f"[DE] Dokładność na zbiorze testowym: {accuracy_de:.2f}%")


[DE] Dokładność na zbiorze testowym: 100.00%
